In [1]:
import pandas as pd

df = pd.read_csv('Crop_recommendation.csv')
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [2]:
df['label'] = pd.Categorical(df['label']).codes
df.head()


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,20
1,85,58,41,21.770462,80.319644,7.038096,226.655537,20
2,60,55,44,23.004459,82.320763,7.840207,263.964248,20
3,74,35,40,26.491096,80.158363,6.980401,242.864034,20
4,78,42,42,20.130175,81.604873,7.628473,262.717340,20


In [3]:
X = df.drop(['label'],axis=1).values
y = df['label'].values

print(X.shape, y.shape)

(2200, 7) (2200,)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_raw, y_train, y_raw = train_test_split(X, y, test_size=0.4)
X_test, X_val, y_test, y_val = train_test_split(X_raw, y_raw, test_size=0.3)

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)
print('Val', X_val.shape, y_val.shape)

Train (1320, 7) (1320,)
Test (616, 7) (616,)
Val (264, 7) (264,)


In [5]:
import keras

inputs = keras.Input((7,))

x = keras.layers.Reshape((7, 1))(inputs)
x = keras.layers.LSTM(48)(x)
x = keras.layers.Dense(24, activation="gelu")(x)
outputs = keras.layers.Dense(22, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 48)             │         9,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         1,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22)             │           550 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,326 (44.24 KB)

 Trainable params: 11,326 (44.24 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test));


Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0732 - loss: 3.0971 - val_accuracy: 0.1218 - val_loss: 3.0311
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1748 - loss: 2.9853 - val_accuracy: 0.2029 - val_loss: 2.8031
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2285 - loss: 2.6583 - val_accuracy: 0.2727 - val_loss: 2.3526
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3276 - loss: 2.2141 - val_accuracy: 0.4334 - val_loss: 1.9198
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4848 - loss: 1.7991 - val_accuracy: 0.5455 - val_loss: 1.6273
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5721 - loss: 1.5147 - val_accuracy: 0.6494 - val_loss: 1.3711
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6727 - loss: 1.3229 - val_accuracy: 0.7224 - val_loss: 1.1901
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7233 - loss: 1.1327 - val_accuracy: 0.8133 - val_loss

In [7]:
y_pred = keras.ops.argmax(model.predict(X_val), 1)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [8]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.97%


In [9]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         7
           2       0.87      1.00      0.93        13
           3       1.00      1.00      1.00        12
           4       1.00      1.00      1.00        12
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00        18
           8       0.80      0.89      0.84         9
           9       1.00      1.00      1.00        17
          10       0.75      1.00      0.86         9
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00        10
          13       1.00      0.78      0.88        23
          14       1.00      1.00      1.00        10
          15       1.00      1.00      1.00        19
          16       1.00      1.00      1.00         7
          17       1.00    

In [10]:
import numpy as np
new_input = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7])

# Reshape the input to match the model's input shape
new_input_reshaped = np.reshape(new_input, (1, 7))

# Predict the output
predicted_output = model.predict(new_input_reshaped)

# Get the class with the highest probability
predicted_class = keras.ops.argmax(predicted_output, axis=1)

print(f"Predicted class: {predicted_class.numpy()[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted class: 13


In [ ]:
print("Hello")